# Визуализация данных

* [Matplotlib](#Matplotlib)
* [Pandas' buildin plotting](#Pandas'-buildin-plotting)
* [Seaborn](#Seaborn)
* [Pandas Styling](#Pandas-Styling)
* [Bokeh](#Bokeh)
* [Holoviews](#Holoviews)
* [Дополнительные материалы](#Дополнительные-материалы)

In [ ]:
import pandas as pd
import numpy as np

Загрузим выборку бриллиантов [diamonds](https://www.kaggle.com/shivam2503/diamonds).
Выборка содержит 54к различных бриллианта с характеристиками (цена, вес, ...).

In [ ]:
diamonds_url = 'http://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/diamonds.csv'

cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']

df = pd.read_csv(diamonds_url, usecols='carat cut color clarity depth table price x y z'.split())

for column, order in [('cut', cut_order),
                      ('clarity', clarity_order),
                      ('color', color_order)]:
    df[column] = df[column].astype('category').cat.set_categories(order, ordered=True)

In [ ]:
df.head()

# [Matplotlib](https://matplotlib.org)

Базовая библиотека по визуализации в python. На её основе сделано много других библиотек, таких как seaborn, pandas'овский `.plot()`, holoviews. Предоставляет обширные возможности, но иногда за счет размера кода. Основное удобное API - [`pyplot()`](http://matplotlib.org/api/pyplot_api.html).

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

У объекта **`ax`** (axes) - оси координат, вызываем отдельные методы (рисование диаграммы рассеивания, подпись оси, ...).  

In [ ]:
fig, ax = plt.subplots()

ax.scatter(x='carat', y='depth', data=df, c='k', alpha=.15)
ax.set_xlabel('carat')
ax.set_ylabel('depth');

Есть интерфейс создания графиков напрямую через вызовы методов pyplot - это API ближе к Matlab.

In [ ]:
x = np.arange(100)

plt.plot(x, np.power(x, 2))
plt.xlabel('X')
plt.ylabel('$X^2$')
plt.grid(True);

# [Pandas' buildin-plotting](https://pandas.pydata.org/pandas-docs/stable/visualization.html)

Встроенный метод **`.plot()`** у объектов **`DataFrame`** и **`Series`** позволяет рисовать большое количество полезных статистических графиков (line, scatter, bar-chart, ...). Очень удобен для исследования "на лету".

In [ ]:
df.plot.scatter(x='carat', y='depth', c='k', alpha=.15);

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

df.plot(x='carat', y='depth', kind='scatter', c='k', alpha=.15, ax=ax1)
df.plot(x='carat', y='price', kind='scatter', c='k', alpha=.15, ax=ax2);

# [Seaborn](http://seaborn.pydata.org)

Высокоуровневое API для посторения статистических графиков на основе matplotlib.

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x='cut', data=df);

In [ ]:
sns.barplot(x='cut', y='price', data=df);

Есть очень удобная диаграмма для сравнения двух параметров.

In [ ]:
sns.jointplot(x='carat', y='price', data=df, size=6, alpha=.25,
              color='k', marker='.');

Есть очень мощный инструмент, рисующий диаграммы рассеивания сразу для всех пар признаков.

In [ ]:
(df.loc[:, ['carat', 'price', 'x', 'cut']]
   .pipe(sns.pairplot, hue='cut'));

Большинсово функций в seaborn возвращают один график (одну систему координат Axes), а вот pairplot - целую сетку таких графиков.

In [ ]:
def core(df, alpha=.05):
    """Take main part of dataset"""
    mask = (df > df.quantile(alpha)).all(1) & (df < df.quantile(1 - alpha)).all(1)
    return df[mask]

In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True, dark=0, light=1, reverse=True)

(df.loc[:, ['carat', 'price', 'x']]
   .pipe(core)
   .pipe(sns.PairGrid)
   .map_upper(plt.scatter, marker='.', alpha=.25)
   .map_diag(sns.kdeplot)
   .map_lower(plt.hexbin, cmap=cmap, gridsize=20)
);

# [Pandas Styling](http://pandas.pydata.org/pandas-docs/stable/style.html)

In [ ]:
(df.sample(5)
   .style
   .highlight_max(subset=['price'])
   .background_gradient(cmap='Greens', subset=['depth'])
   .format('{:.3f}', subset=['x', 'y', 'z']))

# [Bokeh](https://bokeh.pydata.org/en/latest/)

Ориентированна на создание web графики, включающую интерактивность.
Интерактивность в некоторых ситуациях очень важна, например, для рисования карт.
Выборка аэропортов: https://openflights.org/data.html.

In [ ]:
import bokeh.plotting as bp
from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import HoverTool
import math

bp.output_notebook()

In [ ]:
# Скачиваем координаты аэропортов
airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
                       names=['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO',
                              'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST',
                              'Tz database time zone', 'Type', 'Source'],
                       index_col='Airport ID', na_values=[r'\N'])
airports.drop(2033, inplace=True) # not valid coordinate

def mercator(lon, lat):
    x = lon * 20037508.34 / 180
    y = np.log(np.tan((90 + lat) * np.pi / 360)) / (np.pi / 180)
    y = y * 20037508.34 / 180
    y = y - 1
    return x, y

airports['x'], airports['y'] = mercator(airports['Longitude'], airports['Latitude'])

airports.tail(2)

In [ ]:
bound = 20000000 # meters

source = bp.ColumnDataSource(airports.sample(1000).to_dict('list'))

fig = bp.figure(tools='pan, wheel_zoom, reset', x_range=(-bound, bound), y_range=(-bound, bound))

points = fig.circle(x='x', y='y', source=source, color='blue', alpha=0.3,
                    radius=10000, radius_units='data')

fig.circle(x='x', y='y', source=source, color='blue', 
           alpha=0.1, radius=5, radius_units='screen')

fig.add_tools(HoverTool(renderers=[points],
                        tooltips=[('Name','@Name'),
                                  ('City','@City')]))

fig.axis.visible = False
fig.add_tile(STAMEN_TONER)

bp.show(fig)

# [Holoviews](http://holoviews.org)

Высокоуровневое API для создания статистических графиков. Ориентированна на выразительность.

In [ ]:
import holoviews as hv
hv.extension('bokeh')

Есть возможность использовать встроенные высокоуровневые функции.

In [ ]:
dimonds_group = (df.assign(dimonds_count=1)
                   .groupby('cut', as_index=False)
                   .agg({'dimonds_count': 'sum', 'price': 'mean'}))

dimonds_group

In [ ]:
price = hv.Bars(dimonds_group, kdims=['cut'], vdims=['price'])
count = hv.Bars(dimonds_group, kdims=['cut'], vdims=['dimonds_count'])

In [ ]:
price + count

Вызов print() показывает структуру графика (название диаграммы и названия осей).

In [ ]:
print(price)

Визуальные настройки делаются через magic `%%opts`

In [ ]:
%%opts Bars [height=200, width=400]
price

Есть удобная возможность создавать диаграммы с виджетами

In [ ]:
df_grouped = (df.groupby(['cut', 'color'])
                ['price'].mean().reset_index())
df_grouped.sample(3)

Сначала конвертим в объект Dataset, прописывая столбцы-ключи, и стобцы-значения.

In [ ]:
data = hv.Dataset(df_grouped, kdims=['cut', 'color'], vdims=['price'])
data

Тпереь при рисовании диаграмм, те ключи, которые не указаны, станут виджетом.

In [ ]:
data.to(hv.Bars, kdims=['cut'], vdims=['price'])

In [ ]:
diagrams = {i: hv.Image(np.random.randn(10, 10)) for i in range(10)}

hv.HoloMap(diagrams)

In [ ]:
import bokeh.sampledata
bokeh.sampledata.download()

In [ ]:
import sqlite3 as sql
from bokeh.sampledata.movies_data import movie_path


query = '''
SELECT
    imdbID,
    Title,
    Year,
    imdbRating,
    Country,
    Oscars,
    tomatoes.Rating as tomatoesRating,
    Reviews,
    userRating,
    BoxOffice,
    Production
FROM omdb, tomatoes
WHERE omdb.ID = tomatoes.ID
'''

with sql.connect(movie_path) as conn:
    movies = pd.read_sql(query, conn)
    
movies.head()

In [ ]:
hv.extension('matplotlib')

In [ ]:
%%opts NdOverlay [legend_position='bottom' legend_cols=1] NdLayout [vspace=0.5]

(movies.query('Country in ["USA", "France", "UK"]')
       .groupby(['Year', 'Country'])
       .agg({'imdbRating': 'mean',
             'tomatoesRating': 'mean',
             'userRating': 'mean'})
       .stack()
       .reset_index()
       .rename(columns={'level_2': 'type', 0: 'mean_rating'})
       .pipe(hv.Dataset, kdims=['Year', 'type', 'Country'], vdims=['mean_rating'])
       .to(hv.Curve, kdims=['Year'], vdims=['mean_rating'])
       .overlay(dimensions=['type'])
       .layout(dimensions=['Country'])
       .cols(2))

In [ ]:
hv.extension('bokeh')

In [ ]:
%%opts Scatter [tools=['hover'] show_grid=True logy=True color_index='Oscars' colorbar=True colorbar_position='left']
%%opts Scatter (cmap='viridis' size=5) [width=500 height=500]

(movies.query('Year >= 2000')
       .pipe(hv.Dataset, kdims=['imdbRating', 'Year'], vdims=['BoxOffice', 'Oscars', 'Title'])
       .to(hv.Scatter, kdims=['imdbRating'], vdims=['BoxOffice', 'Oscars', 'Title']))

# Дополнительные материалы

* Документации библиотек: 
[matplotlib](https://matplotlib.org);
[seaborn](http://seaborn.pydata.org);
[pandas-visualization](https://pandas.pydata.org/pandas-docs/stable/visualization.html);
[bokeh](https://bokeh.pydata.org/en/latest/);
[holoviews](http://holoviews.org).
* Интерактивные туториалы с [datacamp](https://www.datacamp.com): 
[matplotlib-tutorial](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python); 
[seaborn-tutorial](https://www.datacamp.com/community/tutorials/seaborn-python-tutorial);
[pandas-visualization](https://www.datacamp.com/community/tutorials/pandas-idiomatic#visualization).
* Обзор библиотек по визуализации от Tom Augspurger: 
[visualization](https://tomaugspurger.github.io/modern-6-visualization.html).
* Другие популярные библиотеки:
[Ploty](https://plot.ly/python/); 
[Lightning](http://lightning-viz.org/);
[Glueviz](http://www.glueviz.org/en/stable/);
[vispy](http://vispy.org/);
[bqplot](https://github.com/bloomberg/bqplot).

Доклад Jake VanderPlas с pycon 2017 про стек визуализации на python.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('FytuB8nFHPQ', start=240)